In [1]:
import os

In [2]:
%pwd

'd:\\myprojects\\Chicken_disease_classification\\Chicken-disease-classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\myprojects\\Chicken_disease_classification\\Chicken-disease-classification'

In [5]:
import tensorflow as tf


In [7]:
model=tf.keras.models.load_model('artifacts/training/trained_model.h5')

In [8]:
#evaluation configuration
from dataclasses import dataclass, field
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from chicken_disease_prediction.constants import *
from chicken_disease_prediction.utils.common import read_yaml, create_directories,save_json

In [15]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config



In [16]:
from urllib.parse import urlparse


In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [18]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-06-06 22:42:37,567: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-06 22:42:37,570: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-06 22:42:37,571: INFO: common: created directory at: artifacts]
Found 1002 images belonging to 1 classes.
63/63 [==============================] - 63s 989ms/step - loss: 121.4948 - accuracy: 0.9930
[2025-06-06 22:43:41,083: INFO: common: json file saved at: scores.json]
